# Time scales in astronomy

The formal definition of these scales are maintained by the International Astronomical Union [Standards of Fundamental Astronomy](http://www.iausofa.org/) (SOFA). The document [SOFA Time Scale and Calendar Tools](http://www.iausofa.org/sofa_ts_c.pdf), sections 2 and 3 are particularly helpful if you want a brief, high-level summary of the time scales, and this overview draws heavily on them.

The worked examples use [astropy](https://astropy.org) for the calculations, and some of the images below are from the [astropy time documentation](http://astropy.readthedocs.io/en/stable/time/index.html).

Note that astropy itself uses a lightly modified version of the official SOFA implementations the time transforms (the modified version is called ERFA, and the primary difference between it and the official SOFA code is the license under which it is distributed).

# Big (but confusing) picture

The sheer number of time scales used in astronomy is intimidating; the diagram below, from the astropy documentation, shows all seven of them (the blue boxes):

![Time scales and relationships](https://github.com/astropy/astropy/raw/master/docs/time/time_scale_conversion.png)

A more complete picture, showing all of the possible transforms implemented by the SOFA team is Figure 1 of [SOFA Time Scale and Calendar Tools](http://www.iausofa.org/sofa_ts_c.pdf).


Fortuately, of these time scales there are only three that are likely to matter to many astronomers: Coordinated Universal Time (UTC), Terrestrial Time (TT), and "Earth rotation time" (confusingly also called universal time), UT1.

International Atomic Time (TAI) will also be discussed briefly below because it and TT are virtually the same (that does *NOT* imply they are interchangable).

For completeness, because they won't be discussed further, Geocentric Coordinate Time (TCG) and Barycentric Coordinate Time (TCB) are time scales carefully defined in particular reference frames in the context of general relativity that are convenient for theoretical calculations. Unless you are doing work that requires accuracy to the nearest nanosecond you do not need to worry about them.

Though Barycentric Dynamical Time (TDB) is formally derived from TCB it is, for these purposes, very close to Terrestrial Time (TT).

# Essential time scales

## International Atomic Time (TAI) is the official time standard of the planet

Beginning in the 1980s the formal definition of a second was changed to specific number of periodds of the radiation produced in a specific transitition in a specific isotope of cesium under specific environment conditions.

A day is defined to be exactly 86,400 seconds. 

This time scale is conceptually very simple even though their inner workings are not. An atomic clock is essentially like a clock in a house except that it runs at an extraordinary steady rate. Two identically prepared clocks started now will differ by less than a second a million years from now. In practice, it is an average of the time on a couple of hundred atomic clocks around the world that is used as TAI.

## Earth rotation time (UT1) used to be the official time standard, and it is used to calculate sidereal time

Until the advent of atomic clocks, the official "clock" for thousands of years had been the motion of the Earth.

This seems at first like a conceptually simple standard. As a first try, one might define a day as the time interval between two successive observations of the Sun as it crosses the meridian (i.e. the time interval between two successive "highest point in sky" or even two successive sunrises or sunsets). A year could be defined as the time interval between two successive crossings of the Sun over the Earth's equator moving south to north. 

In this scale, a second is defined to be 1/86400 of one day.

It turns out that a day defined this varies in length over the course of the year (because the Earth's orbit is slightly elliptical, so the speed of the Earth in its orbit varies slightly), and a day defined this way gets longer with time because the rotation rate of the Earth is gradually slowing. In addition, the gravitational pull of the Moon and Sun cause changes in the direction of axis of rotation, which affect both the lengths of days and years.

### You still need to be aware of UT1 because it is essentially a measure of Eath's orientation in space, and your telescope is attached to the Earth

You shouldn't normally need to calculate UT1 yourself, but certainly the computer that points your telescope needs to!

## UTC (Coordinated Universal Time) is what your computer displays (almost)

It runs at the same rate at TAI, but *leap seconds* are added occasionally so that the difference between UTC and UT1 is never larger than 0.9 seconds.

This a compromise between the advantages of an atomic lock (very steady rate) and the convenience of having your clock time remain closely tied to the motions of the sun, at the expense of occasional leap seconds.

UTC behaves oddly at a leap second; in most cases the clock simply pauses for one second.

One way to think about this leap second is that most days in UTC are exactly 86,400 seconds, like in atomic time, but once every 18-24 months one day is 86401 seconds long (and even less frequently a UTC day is 86,399 seconds long, a negative leap second).

The interval between leap seconds is not random but is also not precisely predicatable because it depends on the rate at which the Eath's rotation rate is changing. It is possible to very precisely *measure* that rate, but not to predict it with high enough precision to plan leap seconds more than about 6 months in advance.

Computers typically add an offset for your time zone, so the time displayed on the computer is typically UTC adjusted for the local time zone. For example, Central Standard Time, which covers the central United States, is UTC - 6 hours, and CDT is UTC- 5 hours.


# Terrestrial Time (TT) is atomic time with an offset

Its main practical use is in the ephemerides provided for solar system objects by NASA, which are used to accurately calculate the positions of solar system bodies.

The relationship to atomic time really is very simple; terrestrial time is simply atomic time plus a constant number of seconds, and that constant never changes. Its value is deliberately omitted here to avoid any temptation the reader might have to start writing her own code to carry out time transforms. Though this one case is simple, the rest are not.

In [1]:
from astropy.time import Time

In [16]:
leap_time0 = Time("1994-06-30T23:59:59", scale='utc', format='isot')
leap_time0.mjd

49533.99997685212

In [2]:
leap_time1 = Time("1994-06-30T23:59:60", scale='utc', format='isot')

In [13]:
leap_time1.mjd

49533.99998842606

In [32]:
leap_time1.tai.mjd

49534.00032407408

In [4]:
leap_time2 = Time("1994-07-01T00:00:00", scale='utc', format='isot')

In [14]:
leap_time2.mjd

49534.0

In [23]:
leap_time3 = Time("1994-07-01T00:00:01", scale='utc', format='isot')

In [24]:
leap_time3.mjd

49534.00001157408

In [17]:
leap_time4 = Time("1995-06-30T23:59:60", scale='utc', format='isot')

In [18]:
leap_time4.mjd

49899.0

In [19]:
leap_time4.mjd - leap_time1.mjd

365.00001157393854

In [21]:
leap_time5 = Time("1995-07-01T00:00:00", scale='utc', format='isot')
leap_time5.mjd

49899.0

In [27]:
(leap_time1.mjd - leap_time0.mjd)*86400 - 1

-1.1710450053215027e-05

In [33]:
(leap_time1.tai.mjd - leap_time0.tai.mjd)*86400 - 1

2.337619662284851e-07

In [28]:
(leap_time3.mjd - leap_time2.mjd)*86400 - 1

2.337619662284851e-07

# Yikes! Astropy! Why do you give me a JD for UTC?

See footnote 9, p. 20 of [SOFA Time Scale and Calendar Tools](http://www.iausofa.org/sofa_ts_c.pdf), note above example that demonstrates that the JD on a leap day just is not correct (or sensible).

In [29]:
leap_time1.ut1

<Time object: scale='ut1' format='isot' value=1994-06-30T23:59:59.783>

In [30]:
leap_time2.ut1

<Time object: scale='ut1' format='isot' value=1994-07-01T00:00:00.783>

In [31]:
leap_time2.tai.mjd

49534.00033564815